In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv('co2_emissions_kt_by_country.csv')
df.head()

,country_code,country_name,year,value
0,ABW,Aruba,1960,11092.675
1,ABW,Aruba,1961,11576.719
2,ABW,Aruba,1962,12713.489
3,ABW,Aruba,1963,12178.107
4,ABW,Aruba,1964,11840.743


In [3]:
asia = df[df['country_name'].str.contains('Asia')]['country_name'].unique()
europe = df[df['country_name'].str.contains('Euro')]['country_name'].unique()
caribbean = df[df['country_name'].str.contains('Caribbean')]['country_name'].unique()
africa = df[df['country_name'].str.contains('Africa')]['country_name'].unique() # remove South Africa
africa = africa[africa != 'South Africa']
demographic = df[df['country_name'].str.contains('demo')]['country_name'].unique()
ida = df[df['country_name'].str.contains('IDA')]['country_name'].unique()
ibrd = df[df['country_name'].str.contains('IBRD')]['country_name'].unique()
income = df[df['country_name'].str.contains('income')]['country_name'].unique()
other_regions = ['World', 'Europe', 'North America', 'South Asia', 'OECD members', 'Euro area', 'Arab World', 'Heavily indebted poor countries (HIPC)',
          'Small states','Other small states' , 'Fragile and conflict affected situations', 'Least developed countries: UN classification', 
          'Pacific island small states']
country_groups = np.concatenate((asia, europe, caribbean, africa, demographic, ida, ibrd,income, other_regions))
df_country = df.query("country_name not in @country_groups").copy()

In [5]:
fig = px.choropleth(df_country, locations="country_code",
                    animation_frame="year", animation_group="country_name",
                    color="value", 
                    hover_name="country_name",
                    hover_data=['year', 'country_name', 'value'],
                    color_continuous_scale=px.colors.sequential.dense)
fig.show()

In [7]:
countries = ['United States', 'China','Russian Federation',  'India', 'United Kingdom', 'France', 'Japan', 'Canada',  'Italy', 'Germany']

df_country['country_category'] = np.where(df_country['country_name'].isin(countries), df_country['country_name'], 'Other')

df_country_10 = df_country.groupby(['year','country_category'], as_index=False)['value'].sum().copy()

In [8]:
px.area(df_country_10, x='year', y='value', line_group='country_category', color='country_category',
                   category_orders={
                    "country_category": ['Other'] + countries 
            },
                labels={
                     "value": "CO2 Emissions",
                     "year": "Year",
                     "country_name": "Country"
                 },
              title = 'Annual CO2 Emissions, by Country',
       template="simple_white")